<a href="https://colab.research.google.com/github/lalitjoshi007/Speech_to_text_project/blob/main/Speech_to__text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn transformers ffmpeg-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00


In [2]:
!pip install python-multipart


In [3]:
!pip install pyngrok

In [4]:
from pyngrok import ngrok


ngrok.set_auth_token("2nFBM9vPrW801lhj6grLA9J5br4_4PMev68GypN1FEbekY5aY")


In [8]:
import os
import nest_asyncio
import uvicorn
import ffmpeg
from fastapi import FastAPI, UploadFile, File
from transformers import pipeline
from pyngrok import ngrok
import threading


nest_asyncio.apply()


app = FastAPI()


whisper_model = pipeline(model="openai/whisper-small", task="automatic-speech-recognition")


def convert_mp3_to_wav(mp3_path):
    wav_path = mp3_path.replace(".mp3", ".wav")
    try:
        ffmpeg.input(mp3_path).output(wav_path).run(overwrite_output=True)
        print(f"Converted {mp3_path} → {wav_path}")
        return wav_path
    except ffmpeg.Error as e:
        print("FFmpeg error:", e)
        return None


def transcribe_audio(wav_path):
    return whisper_model(wav_path)['text']


@app.post("/transcribe/")
async def transcribe(file: UploadFile = File(...)):
    mp3_path = f"audio_{file.filename}"
    with open(mp3_path, "wb") as f:
        f.write(await file.read())

    wav_path = convert_mp3_to_wav(mp3_path)
    if wav_path is None:
        return {"error": "Audio conversion failed"}

    result = transcribe_audio(wav_path)

    os.remove(mp3_path)
    os.remove(wav_path)

    return {"transcription": result}


def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server)
thread.start()


public_url = ngrok.connect(8000)
print(f"Your app is live at: {public_url}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cpu
INFO:     Started server process [488]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Your app is live at: NgrokTunnel: "https://afed-35-227-121-38.ngrok-free.app" -> "http://localhost:8000"
